This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [19]:
# If haven't downloaded it yet, please get the data file with wget
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [21]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df=pd.read_csv('data/data_000637.txt',delimiter=',')[0:30000]
df

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
29995,1,1,60,3869200466,356,11
29996,1,0,64,3869200466,347,4
29997,1,0,139,3869200466,357,0
29998,1,0,61,3869200466,353,10


2\. Estimate the number of BX in a ORBIT (the value 'x').

In [22]:
x = df["BX_COUNTER"].max()+1
print("The number of BX in a ORBIT is : ",x)

The number of BX in a ORBIT is :  3564


3\. Find out the duration of the data taking in hours, minutes and seconds. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading the whole dataset.

In [23]:
conversion = 25*10**(-9)
deltatime = (df['ORBIT_CNT'].max()-df['ORBIT_CNT'].min())*x*conversion
print('Time: ', deltatime)

Time:  0.026640900000000002


4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition.

In [24]:
df['orbit_respect_0']=df['ORBIT_CNT'] - df['ORBIT_CNT'].min()
df['time'] = conversion*x*df['orbit_respect_0'] + df['BX_COUNTER']*conversion + df['TDC_MEAS']*25/30*1e-9 
df.head()

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,orbit_respect_0,time
0,1,0,123,3869200167,2374,26,0,0.000059
1,1,0,124,3869200167,2374,27,0,0.000059
2,1,0,63,3869200167,2553,28,0,0.000064
3,1,0,64,3869200167,2558,19,0,0.000064
4,1,0,64,3869200167,2760,25,0,0.000069


5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [25]:
df = df.groupby(by='TDC_CHANNEL', sort=True).count()
df = df.sort_values('HEAD', ascending=False)
top = [x for x in df.index[0:3]]
print("top 3 TDC channels are ", top)

top 3 TDC channels are  [139, 64, 63]


6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit). Count also the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [31]:
orbits = df.drop_duplicates('ORBIT_CNT')
print('umber of non-empty orbits:', len(orbits))

orbits139 = df[df['TDC_CHANNEL'] == 139].drop_duplicates('ORBIT_CNT')
print('number of unique orbits with at least one measurement from TDC_CHANNEL=139:', len(orbits139))

umber of non-empty orbits: 97


KeyError: 'TDC_CHANNEL'

7\. **Optional:** Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel

In [32]:
fpga0 = df[df['FPGA'] == 0]
fpga0.hist(column='TDC_CHANNEL', bins=fpga0['TDC_CHANNEL'].max(), color='green')
plt.title('FPGA0 occupancy plot')
plt.xlabel('TDC channel')
plt.ylabel('# counts')


fpga1 = df[df['FPGA'] == 1]
fpga1.hist(column='TDC_CHANNEL', bins=fpga1['TDC_CHANNEL'].max(), color='blue')
plt.title('FPGA1 occupancy plot')
plt.xlabel('TDC channel')
plt.ylabel('# counts')

KeyError: 'TDC_CHANNEL'